<a href="https://colab.research.google.com/github/qurrata111/OpenNMT-py/blob/master/docs/source/examples/OpenNMT_py_2_QuickStart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

QucikStart for OpenNMT-py 2 as found at:
https://github.com/OpenNMT/OpenNMT-py#quickstart

In [1]:
# Install OpenNMT-py 2.x
# NOTE: By the end of the insatallation, it might ask for restarting the runtime...
# In this case, just click the "RESTART RUNTIME" button.

!pip3 install OpenNMT-py

     |████████████████████████████████| 216 kB 5.4 MB/s 
     |████████████████████████████████| 73 kB 1.6 MB/s 
     |████████████████████████████████| 15.8 MB 136 kB/s 
     |████████████████████████████████| 56 kB 4.1 MB/s 
     |████████████████████████████████| 1.2 MB 29.8 MB/s 
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.10.0
    Uninstalling torchtext-0.10.0:
      Successfully uninstalled torchtext-0.10.0


In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


In [3]:
import torch
print(torch.__version__)
print(torch.version.cuda)

1.9.0+cu102
10.2


In [4]:
# Download the files of the QuickStart

!wget https://s3.amazonaws.com/opennmt-trainingdata/toy-ende.tar.gz
!tar xf toy-ende.tar.gz

--2021-09-22 15:05:06--  https://s3.amazonaws.com/opennmt-trainingdata/toy-ende.tar.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.22.45
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.22.45|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1662081 (1.6M) [application/x-gzip]
Saving to: ‘toy-ende.tar.gz’

toy-ende.tar.gz     100%[===================>]   1.58M  8.47MB/s    in 0.2s    

2021-09-22 15:05:06 (8.47 MB/s) - ‘toy-ende.tar.gz’ saved [1662081/1662081]



In [5]:
# Optional: List the extracted files

!ls toy-ende/

src-test.txt   src-val.txt   tgt-train.txt
src-train.txt  tgt-test.txt  tgt-val.txt


In [6]:
# Optional: Print the first 3 lines of the source file

!head -n 3 toy-ende/src-train.txt

It is not acceptable that , with the help of the national bureaucracies , Parliament &apos;s legislative prerogative should be made null and void by means of implementing provisions whose content , purpose and extent are not laid down in advance .
Federal Master Trainer and Senior Instructor of the Italian Federation of Aerobic Fitness , Group Fitness , Postural Gym , Stretching and Pilates; from 2004 , he has been collaborating with Antiche Terme as personal Trainer and Instructor of Stretching , Pilates and Postural Gym .
&quot; Two soldiers came up to me and told me that if I refuse to sleep with them , they will kill me . They beat me and ripped my clothes .


In [7]:
# Optional: Check the number of lines in the source file

!echo "Number of lines:" && wc -l toy-ende/src-train.txt

Number of lines:
10000 toy-ende/src-train.txt


In [8]:
# Create the YAML configuration file
# On a regular machine, you can create it manually or with nano

config = '''# toy_en_de.yaml

## Where the samples will be written
save_data: toy-ende/run/example

## Where the vocab(s) will be written
src_vocab: toy-ende/run/example.vocab.src
tgt_vocab: toy-ende/run/example.vocab.tgt

## Where the model will be saved
save_model: model/model

# Prevent overwriting existing files in the folder
overwrite: False

# Corpus opts:
data:
    corpus_1:
        path_src: toy-ende/src-train.txt
        path_tgt: toy-ende/tgt-train.txt
    valid:
        path_src: toy-ende/src-val.txt
        path_tgt: toy-ende/tgt-val.txt

world_size: 1
gpu_ranks: [0]

# Remove or modify these lines for bigger files
train_steps: 3000
valid_steps: 300

early_stopping: 5
'''

with open("toy_en_de.yaml", "w+") as config_yaml:
  config_yaml.write(config)

!cat toy_en_de.yaml

# toy_en_de.yaml

## Where the samples will be written
save_data: toy-ende/run/example

## Where the vocab(s) will be written
src_vocab: toy-ende/run/example.vocab.src
tgt_vocab: toy-ende/run/example.vocab.tgt

## Where the model will be saved
save_model: model/model

# Prevent overwriting existing files in the folder
overwrite: False

# Corpus opts:
data:
    corpus_1:
        path_src: toy-ende/src-train.txt
        path_tgt: toy-ende/tgt-train.txt
    valid:
        path_src: toy-ende/src-val.txt
        path_tgt: toy-ende/tgt-val.txt

world_size: 1
gpu_ranks: [0]

# Remove or modify these lines for bigger files
train_steps: 3000
valid_steps: 300

early_stopping: 5


In [9]:
# Build Vocabulary

!onmt_build_vocab -config toy_en_de.yaml -n_sample -1

Corpus corpus_1's weight should be given. We default it to 1 for you.
[2021-09-22 15:05:34,657 INFO] Counter vocab from -1 samples.
[2021-09-22 15:05:34,657 INFO] n_sample=-1: Build vocab on full datasets.
[2021-09-22 15:05:34,667 INFO] corpus_1's transforms: TransformPipe()
[2021-09-22 15:05:34,969 INFO] Counters src:24995
[2021-09-22 15:05:34,969 INFO] Counters tgt:35816


In [10]:
# Check if GPU is active
# If not, go to "Runtime" menu > "Change runtime type" > "GPU"

!nvidia-smi -L

GPU 0: Tesla K80 (UUID: GPU-6606fc0f-8e3e-30d4-cd48-7d02aca975ef)


In [11]:
# Make sure the GPU is visable to PyTorch

import torch

gpu_id = torch.cuda.current_device()
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(gpu_id))

True
Tesla K80


In [12]:
# Train the NMT model

!onmt_train -config toy_en_de.yaml

[2021-09-22 15:05:47,578 INFO] Missing transforms field for corpus_1 data, set to default: [].
[2021-09-22 15:05:47,579 WARNING] Corpus corpus_1's weight should be given. We default it to 1 for you.
[2021-09-22 15:05:47,579 INFO] Missing transforms field for valid data, set to default: [].
[2021-09-22 15:05:47,579 INFO] Parsed 2 corpora from -data.
[2021-09-22 15:05:47,579 INFO] Get special vocabs from Transforms: {'src': set(), 'tgt': set()}.
[2021-09-22 15:05:47,579 INFO] Loading vocab from text file...
[2021-09-22 15:05:47,579 INFO] Loading src vocabulary from toy-ende/run/example.vocab.src
[2021-09-22 15:05:47,633 INFO] Loaded src vocab has 24995 tokens.
[2021-09-22 15:05:47,647 INFO] Loading tgt vocabulary from toy-ende/run/example.vocab.tgt
[2021-09-22 15:05:47,754 INFO] Loaded tgt vocab has 35816 tokens.
[2021-09-22 15:05:47,774 INFO] Building fields with vocab in counters...
[2021-09-22 15:05:47,835 INFO]  * tgt vocab size: 35820.
[2021-09-22 15:05:47,871 INFO]  * src vocab siz

In [13]:
# Optional: Make a smaller file with 50 sentences instead of 2737
!head -20 toy-ende/src-test.txt > toy-ende/src-test-20.txt

!wc -l toy-ende/src-test.txt
!wc -l toy-ende/src-test-20.txt

2737 toy-ende/src-test.txt
20 toy-ende/src-test-20.txt


In [15]:
# Translate

!onmt_translate -model model/model_step_3000.pt -src toy-ende/src-test.txt -output toy-ende/pred_1000.txt -gpu 0 -verbose

Streaming output truncated to the last 5000 lines.
SENT 1738: ['Pope', 'Francis', 'to', 'name', 'first', 'cardinals', 'in', 'February']
PRED 1738: Anatomie seiner ersten Mission von 12 Jahren 40 mm in Tirol .
PRED SCORE: -26.6673

[2021-09-22 15:30:41,943 INFO] 
SENT 1739: ['Pope', 'Francis', 'will', 'create', 'new', 'cardinals', 'of', 'the', 'Catholic', 'Church', 'for', 'his', 'first', 'time', 'on', 'February', '22', ',', 'the', 'Vatican', 'announced', 'Thursday', '.']
PRED 1739: Mit der Verwendung von zwei Jahren wird die internationale Anführungszeichen für seine Hotelreservierung , die von der Allianz von 0 bis 1 Jahren in der ersten Phase von 10 Prozent der ersten Seite in der ganzen Welt eingesetzt wurde , die von der EBRD , die in der ganzen Seite von 10 Jahren in der ersten Phase eingesetzt wurde .
PRED SCORE: -126.8562

[2021-09-22 15:30:41,943 INFO] 
SENT 1740: ['Cardinals', 'are', 'the', 'highest-ranking', 'clergy', 'in', 'the', 'Catholic', 'Church', 'below', 'the', 'pope', 